# Module 1. Basic Usage
---

### AWS Chalice란?
Chalice는 AWS의 오픈 소스 서버리스 프레임워크로 빠르고 쉽게 서버리스 어플리케이션을 구축할 수 있습니다.
Flask 스타일의 마이크로 웹 프레임워크를 기반으로 하고 있으며, 자동으로 AWS Lambda 함수를 생성하고 API Gateway 엔드포인트를 구성해 줍니다.
또한 Amazon DynamoDB, Amazon S3, SQS, SNS 등과 같은 서비스의 통합도 지원하기에 ML엔지니어나 개발자들이 핵심 로직 개발에 집중할 수 있습니다.

## Hello world

In [65]:
# !pip install chalice
# !sudo yum -y install tree

In [53]:
PROJECT = "hello-world"
!rm -rf $PROJECT
!chalice new-project $PROJECT

Your project has been generated in ./hello-world


In [49]:
!ls $PROJECT

app.py	__pycache__  requirements.txt


In [54]:
!tree -a $PROJECT/.

hello-world/.
├── app.py
├── .chalice
│   └── config.json
├── .gitignore
└── requirements.txt

1 directory, 4 files


In [55]:
!cat $PROJECT/app.py

from chalice import Chalice

app = Chalice(app_name='hello-world')


@app.route('/')
def index():
    return {'hello': 'world'}


# The view function above will return {"hello": "world"}
# whenever you make an HTTP GET request to '/'.
#
# Here are a few more examples:
#
# @app.route('/hello/{name}')
# def hello_name(name):
#    # '/hello/james' -> {"hello": "james"}
#    return {'hello': name}
#
# @app.route('/users', methods=['POST'])
# def create_user():
#     # This is the JSON body the user sent in their POST request.
#     user_as_json = app.current_request.json_body
#     # We'll echo the json body back to the user in a 'user' key.
#     return {'user': user_as_json}
#
# See the README documentation for more examples.
#


In [56]:
!cd $PROJECT && chalice local --port=8100

Restarting local dev server.
Serving on http://127.0.0.1:8100
127.0.0.1 - - [14/Mar/2023 07:43:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2023 07:43:14] "GET / HTTP/1.1" 200 -
^C


In [14]:
# curl localhost:8100

## Hello, [YOUR_NAME]

In [57]:
%%writefile $PROJECT/app.py

from chalice import Chalice
app = Chalice(app_name='hello-world')

@app.route('/')
def index():
    return {'hello': 'world'}

@app.route('/your_name/{name}')
def index(name):
    return {'hello': name}

@app.schedule(Rate(5, unit=Rate.MINUTES))
def periodic_task(event):
    print(f"{datetime.now()}: I'm running!")


Overwriting hello-world/app.py


In [58]:
!cd $PROJECT && chalice local --port=8100

Restarting local dev server.
Serving on http://127.0.0.1:8100
127.0.0.1 - - [14/Mar/2023 07:44:41] "GET /your_name/daekeun HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2023 07:44:45] "GET /your_name/gildong HTTP/1.1" 200 -
^C


In [17]:
# curl localhost:8100/your_name/daekeun
# curl localhost:8100/your_name/gildong

In [59]:
!cd $PROJECT && chalice deploy

Creating deployment package.
Creating IAM role: hello-world-dev
Creating lambda function: hello-world-dev
Creating Rest API
Resources deployed:
  - Lambda ARN: arn:aws:lambda:us-east-1:143656149352:function:hello-world-dev
  - Rest API URL: https://e91nwvq7rc.execute-api.us-east-1.amazonaws.com/api/


You now have an API up and running using API Gateway and Lambda:

In [60]:
response = client.get_rest_apis(limit=2)

In [61]:
from IPython.core.display import display, HTML
import boto3
client = boto3.client('apigateway')

def make_console_link(region, restapi_id, prefix=None):
    url = f"https://{restapi_id}.execute-api.{region}.amazonaws.com/api/"
    if prefix is not None:
        url += prefix
    link = f"<a target='blank' href={url}>{url}</a>"
    return link  

region = boto3.Session().region_name
response = client.get_rest_apis(limit=2)
restapi_id = response['items'][0]['id']

url = make_console_link(region, restapi_id, "your_name/gildong")
display(HTML(url))

url = make_console_link(region, restapi_id, "your_name/daekeun")
display(HTML(url))

In [62]:
!cd $PROJECT && chalice delete

Deleting Rest API: e91nwvq7rc
Deleting function: arn:aws:lambda:us-east-1:143656149352:function:hello-world-dev
Deleting IAM role: hello-world-dev
